In [14]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap


In [15]:
intercountry_df = pd.read_csv('../Data_sinan/intercountry_tone_count_table', dtype={'EventRootCode': np.str})
world_borders = json.load(open('../Data/world_borders.topojson.json'))
fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]


In [16]:
intercountry_df = intercountry_df[pd.notnull(intercountry_df['LocEvent'])]
intercountry_df = (intercountry_df[intercountry_df["EventRootCode"]!='--'])


In [17]:
intercountry_df = fips_codes.merge(intercountry_df, left_on='fips', right_on='LocEvent', how='inner')
intercountry_df.loc[intercountry_df['country']=='United States', 'country'] = 'United States of America'
intercountry_df.loc[intercountry_df['country']=='Serbia', 'country'] = 'Republic of Serbia'


In [20]:
intercountry_df.shape

(530764, 7)

In [5]:
intercountry_nb_mentions_df = intercountry_df[['country', 'EventRootCode', 'NumberOfInterMentions']]
intercountry_nb_mentions_df = intercountry_nb_mentions_df.groupby(['country', 'EventRootCode']).sum().reset_index(level=[0,1])
countries_fight = intercountry_nb_mentions_df.groupby('country').filter(lambda df: any(df.EventRootCode == '19'))


In [6]:
fights_df = intercountry_df[intercountry_df['EventRootCode']=='19']
fights_df = fights_df[['country', 'NumberOfInterMentions']]
fights_df = fights_df.groupby('country', as_index=False).sum().rename(index=str, columns={"NumberOfInterMentions": "Number of fights"})
fights_df.head(5)


,country,Number of fights
0,Afghanistan,2127414
1,Albania,120746
2,Algeria,102791
3,American Samoa,957
4,Andorra,2735


In [7]:
countries_fight_total_events = countries_fight.groupby('country', as_index=False).sum().rename(index=str, columns={"NumberOfInterMentions": "Total number of event"})
countries_fight_total_events.head(5)


,country,Total number of event
0,Afghanistan,9643195
1,Albania,3352564
2,Algeria,1438293
3,American Samoa,29730
4,Andorra,84974


In [8]:
fights_df = fights_df.merge(countries_fight_total_events, on='country', how='inner')
fights_df['Relative number of fights'] = fights_df['Number of fights']/fights_df['Total number of event']
fights_df.head()

,country,Number of fights,Total number of event,Relative number of fights
0,Afghanistan,2127414,9643195,0.220613
1,Albania,120746,3352564,0.036016
2,Algeria,102791,1438293,0.071467
3,American Samoa,957,29730,0.032190
4,Andorra,2735,84974,0.032186


In [9]:
world_map = folium.Map(tiles='Mapbox bright', zoom_start=5.5)

# Defining a style function to color the map and the borders
style_func=lambda feature: {'fillColor': '#990033','color' : 'black','weight' : 2 }

# Using a GeoJsonTooltip to display cantons' name when mouse is over them
tool=folium.GeoJsonTooltip(fields=['name'], aliases=['Country:'], sticky=True, \
                           style="font-family: Arial; color: black;", \
                           opacity=0.8, \
                           direction='top' \
                          )

# Load border information on the map
folium.TopoJson(world_borders,\
                object_path='objects.countries1',\
                style_function=style_func,\
                tooltip=tool, \
                overlay=False \
               ) \
      .add_to(world_map)

In [10]:
fights_df_dict = fights_df.set_index('country')['Relative number of fights'].to_dict()

color_scale = LinearColormap(['yellow','red'], vmin = min(fights_df_dict.values()), \
                                                vmax = max(fights_df_dict.values()) \
                            )

def color_function(feature):
    value = fights_df_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)
    
folium.TopoJson(
    world_borders,
    object_path='objects.countries1',
    style_function=lambda feature: {
        'fillColor': color_function(feature),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5',
        'fillOpacity' : 0.8
        },
    tooltip=folium.GeoJsonTooltip(fields=['name'],aliases=['Country: '], sticky=True, style="font-family: Arial; color: black;", \
                           opacity=0.8, direction='top'),
    name = 'Country Media Average',
    ).add_to(world_map)

color_scale.caption = 'Relative fight numbers [%]'
world_map.add_child(color_scale)
folium.LayerControl().add_to(world_map)
world_map  